In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install ultralytics --quiet
!pip install deep_sort_realtime --quiet
!pip install motmetrics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [1]:
import os
import cv2
import numpy as np
import torch
from torchvision.transforms import functional as F
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
from IPython.display import Video, display
import motmetrics as mm
import numpy as np
import os
import pandas as pd


## Folder

In [ ]:
# Configuration
INPUT_FRAMES_DIR = "/content/drive/My Drive/SportsMOT/test/img1"
OUTPUT_VIDEO_PATH = "/content/drive/My Drive/SportsMOT/output_video_Deepsort_test.mp4"
MODEL_WEIGHTS_PATH = "/content/drive/My Drive/SportsMOT/best.pt"
RESULTS_DIR = "/content/drive/My Drive/SportsMOT/"
RESULTS_TEXT = "tracking_results_deepsort.txt"
os.makedirs(RESULTS_DIR, exist_ok=True)
CLASS_NAMES = ["player", "referee", "ball"]  # Update with your class names
CLASS_COLORS = {
    "player": (255, 255, 0),    # Green
    "referee": (0, 0, 255),   # Red
    "ball": (255, 0, 255)       # Blue
}
MIN_CONFIDENCE = 0.5          # Minimum detection confidence
FRAME_SIZE = (1280, 720)      # Should match your frame size
FPS = 25                      # Adjust based on your video

In [ ]:
tracker = DeepSort(max_age=5, n_init=1, nms_max_overlap=0.2)

In [ ]:
# Load YOLOv8 model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
yolo_model = YOLO(MODEL_WEIGHTS_PATH).to(device)

In [ ]:
# Get sorted list of frame files
frame_files = sorted([f for f in os.listdir(INPUT_FRAMES_DIR) if f.endswith(('.jpg', '.png'))],
                     key=lambda x: int(x.split('.')[0]))

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, FPS, FRAME_SIZE)

In [ ]:
CLASS_COLORS = {
    "ball": (0,200,200),
    "player": (255,255,0),
    "referee": (0,0,255),
}

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 output
video_writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, FPS, FRAME_SIZE)

# Process frames and track objects
for frame_idx, frame_file in enumerate(frame_files):

    frame_path = os.path.join(INPUT_FRAMES_DIR, frame_file)
    frame = cv2.imread(frame_path)
    frame = cv2.resize(frame, FRAME_SIZE)  # Ensure 720p resolution

    # Run YOLO model for detection
    results = yolo_model(frame, device=device, verbose=False)[0]  # Ensure it's the first result

    processed_boxes = []
    if results.boxes is not None:
        boxes = results.boxes.xyxy.cpu().numpy()  # Ensure correct format
        confidences = results.boxes.conf.cpu().numpy()

        # Filter low-confidence detections
        for i, box in enumerate(boxes):
            if confidences[i] < 0.4:  # Confidence threshold
                continue
            xmin, ymin, xmax, ymax = map(int, box)
            width, height = xmax - xmin, ymax - ymin
            processed_boxes.append([[xmin, ymin, width, height], confidences[i]])

    # Update DeepSORT tracker
    tracks = tracker.update_tracks(processed_boxes, frame=frame)
    with open(os.path.join(RESULTS_DIR, RESULTS_TEXT), "a") as f:
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            class_name = track.det_class
            color = CLASS_COLORS.get(class_name, (255, 255, 255))

            frame_num = frame_idx + 1  # You need to track this
            track_id = track.track_id
            x1, y1, x2, y2 = track.to_ltrb()
            conf = str(track.det_conf)[:4]

            line = f"{frame_num},{track_id},{x1},{y1},{x2-x1},{y2-y1},{conf},-1,-1,-1\n"
            f.write(line)

            x1, y1, w, h = map(int, track.to_ltwh())  # Convert to correct format
            x2, y2 = x1 + w, y1 + h
            label = f"{track_id}. {class_name} ({conf})"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

    # Write frame to video
    video_writer.write(frame)

video_writer.release()
print("Processing completed!")

Processing completed!


In [ ]:
def compute_mot_metrics(gt_file, ts_file, seqmap_file=None):

    # Load ground truth and tracking results
    gt = mm.io.load_motchallenge(gt_file)
    ts = mm.io.load_motchallenge(ts_file)

    # Calculate metrics
    acc = mm.utils.compare_to_groundtruth(gt, ts, 'iou', distth=0.5)
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=[
        'mota', 'precision', 'recall', 'motp', 'idf1', 'idp', 'idr',
        'num_switches', 'mostly_tracked',
        'partially_tracked', 'mostly_lost'
    ], name='acc')

    summary_df = pd.DataFrame(summary).transpose()

    return summary_df

# Usage example
gt_path = "/content/drive/My Drive/SportsMOT/test/gt/corrected_gt.txt"
results_path = "/content/drive/My Drive/SportsMOT/tracking_results_deepsort.txt"
metrics_summary = compute_mot_metrics(gt_path, results_path)
print(metrics_summary)

                         acc
mota                0.589685
precision           0.903100
recall              0.669843
motp                0.178865
idf1                0.545130
idp                 0.640045
idr                 0.474731
num_switches       40.000000
mostly_tracked      7.000000
partially_tracked  11.000000
mostly_lost         3.000000


# Video

In [7]:
import os
import cv2
import torch
import numpy as np

# These should be replaced with the actual imports from your YOLO and DeepSORT modules.
# For example:
# from your_yolo_module import YOLO
# from your_deepsort_module import DeepSort

class VideoTracker:
    def __init__(self,
                 input_video_path: str,
                 output_video_path: str,
                 model_weights_path: str,
                 results_dir: str,
                 results_text: str = "tracking_results_deepsort.txt",
                 class_names: list = None,
                 class_colors: dict = None,
                 min_confidence: float = 0.5,
                 frame_size: tuple = (1280, 720),
                 fps: int = 25,
                 device: str = None):
        """
        Initializes the VideoTracker with configuration settings.
        """
        self.input_video_path = input_video_path
        self.output_video_path = output_video_path
        self.model_weights_path = model_weights_path
        self.results_dir = results_dir
        self.results_text = results_text
        self.min_confidence = min_confidence
        self.frame_size = frame_size
        self.fps = fps

        # Set up class names and colors
        self.class_names = class_names if class_names is not None else ["player", "referee", "ball"]
        self.class_colors = class_colors if class_colors is not None else {
            "player": (255, 255, 0),
            "referee": (0, 0, 255),
            "ball": (0, 200, 200)
        }

        # Ensure results directory exists
        os.makedirs(self.results_dir, exist_ok=True)

        # Device configuration
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')

        # Load YOLO model and move it to the appropriate device
        self.yolo_model = YOLO(self.model_weights_path).to(self.device)

        # Initialize DeepSORT tracker
        self.tracker = DeepSort(max_age=5, n_init=1, nms_max_overlap=0.2)

        # Prepare results text file (overwrite if exists)
        self.results_file_path = os.path.join(self.results_dir, self.results_text)
        with open(self.results_file_path, "w") as f:
            f.write("")  # Clear previous content

    def process_video(self):
        """
        Processes the input video frame-by-frame, performs detection and tracking,
        writes tracking results to a text file, and outputs a new video with annotated tracking.
        """
        # Open the video file
        cap = cv2.VideoCapture(self.input_video_path)
        if not cap.isOpened():
            print(f"Error: Could not open video {self.input_video_path}")
            return

        # Optionally, update FPS and frame size from the video properties if needed
        # self.fps = cap.get(cv2.CAP_PROP_FPS)
        # width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        # height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        # self.frame_size = (width, height)

        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(self.output_video_path, fourcc, self.fps, self.frame_size)

        frame_idx = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break  # End of video

            # Resize frame if necessary
            frame = cv2.resize(frame, self.frame_size)

            # Run YOLO model for detection
            results = self.yolo_model(frame, device=self.device, verbose=False)[0]

            # Build a detections list where each entry is:
            # [ [xmin, ymin, width, height], confidence ]
            detections = []
            if results.boxes is not None:
                boxes = results.boxes.xyxy.cpu().numpy()  # Format: [xmin, ymin, xmax, ymax]
                confidences = results.boxes.conf.cpu().numpy()

                # Filter low-confidence detections and prepare boxes for DeepSORT
                for i, box in enumerate(boxes):
                    if confidences[i] < self.min_confidence:
                        continue
                    xmin, ymin, xmax, ymax = map(int, box)
                    width, height = xmax - xmin, ymax - ymin
                    # Append in the expected nested format
                    detections.append([[xmin, ymin, width, height], confidences[i]])


            tracks = self.tracker.update_tracks(detections, frame=frame)

            # Open results file in append mode and write tracking results
            with open(self.results_file_path, "a") as f:
                for track in tracks:
                    if not track.is_confirmed():
                        continue

                    track_id = track.track_id
                    class_name = track.det_class if hasattr(track, 'det_class') else "unknown"
                    color = self.class_colors.get(class_name, (255, 255, 255))
                    frame_num = frame_idx + 1
                    x1, y1, x2, y2 = track.to_ltrb()  # Left, Top, Right, Bottom

                    # Use the confidence score passed from the detection.
                    # Depending on your DeepSORT implementation, the confidence might be stored as 'det_conf' or 'score'.
                    conf_value = (track.det_conf if hasattr(track, 'det_conf') and track.det_conf is not None
                                  else (track.score if hasattr(track, 'score') else 0.0))
                    conf_str = f"{conf_value:.2f}"

                    # Write a line to the results file (customize the format as needed)
                    line = f"{frame_num},{track_id},{x1},{y1},{x2 - x1},{y2 - y1},{conf_str},-1,-1,-1\n"
                    f.write(line)

                    # Draw bounding box and label on the frame
                    x1_ltwh, y1_ltwh, w, h = map(int, track.to_ltwh())  # Left, Top, Width, Height
                    x2_ltwh, y2_ltwh = x1_ltwh + w, y1_ltwh + h
                    label = f"{track_id}. {class_name} ({conf_str})"
                    cv2.rectangle(frame, (x1_ltwh, y1_ltwh), (x2_ltwh, y2_ltwh), color, 2)
                    cv2.putText(frame, label, (x1_ltwh, y1_ltwh - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Write annotated frame to output video
            video_writer.write(frame)
            frame_idx += 1

            # Optionally, display the frame (useful for debugging)
            # cv2.imshow("Tracking", frame)
            # if cv2.waitKey(1) & 0xFF == ord('q'):
            #     break

        # Release resources
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()
        print("Processing completed!")


In [9]:
# Example configuration paths; update them to your local paths as needed.
INPUT_VIDEO_PATH = "/content/drive/My Drive/SportsMOT/val_video.mp4"
OUTPUT_VIDEO_PATH = "/content/drive/My Drive/SportsMOT/Deepsort_test3.mp4"
MODEL_WEIGHTS_PATH = "/content/drive/My Drive/SportsMOT/best.pt"
RESULTS_DIR = "/content/drive/My Drive/SportsMOT/"

# Initialize and run the video tracker
video_tracker = VideoTracker(
    input_video_path=INPUT_VIDEO_PATH,
    output_video_path=OUTPUT_VIDEO_PATH,
    model_weights_path=MODEL_WEIGHTS_PATH,
    results_dir=RESULTS_DIR,
    results_text="tracking_results_deepsort_test3.txt",
    class_names=["player", "referee", "ball"],
    class_colors={
        "player": (255, 255, 0),
        "referee": (0, 0, 255),
        "ball": (0, 200, 200)
    },
    min_confidence=0.6,
    frame_size=(1280, 720),
    fps=25
)
video_tracker.process_video()

0.4197095
0.43135095
0.2929616
0.30777073
0.29193318
0.5819921
0.42822853
0.32581773
0.5380466
0.38887408
0.58681077
0.58630705
0.5477887
0.33716324
0.5312546
0.5484758
0.4276119
0.3256879
0.5901515
0.44657636
0.5644242
0.27396992
0.58073676
0.27116233
0.4749948
0.5902271
0.53943187
0.28322405
0.29237664
0.4896696
0.46537006
0.5964763
0.40997365
0.58495045
0.45915464
0.3102826
0.5345808
0.31596968
0.34848672
0.46936348
0.2573932
0.2589684
0.3986963
0.39947644
0.5799786
0.5752865
0.27773556
0.28615484
0.3753249
0.45267406
0.35024384
0.29064927
0.5612895
0.4537589
0.48332566
0.5997306
0.30687627
0.41506106
0.3511913
0.36381295
0.46545485
0.36579612
0.25455981
0.46428975
0.35503697
0.2543344
0.2665312
0.27504358
0.29762483
0.5695777
0.38936582
0.35039425
0.36988258
0.3468441
0.30030322
0.29216012
0.342135
0.26230633
0.38856885
0.39515698
0.34158197
0.48937005
0.3398411
0.55049324
0.34926674
0.5812318
0.3683327
0.2754229
0.2898261
0.2716924
0.5530433
0.5154727
0.4156447
0.46981695
0.446701